## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import pandas as pd

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Load RetinaNet model

In [2]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('/data/a2301133/RetinaNet/snapshots/', 'resnet50_csv_50.h5')

# load retinanet model
# model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.load_model(model_path, backbone_name='resnet50', convert=True)

# print(model.summary())

In [3]:
# load label to names mapping for visualization purposes
# labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
labels_to_names = {}
with open("SF_mapping.txt", "rb") as f:
    for line in f:
        data = line.strip().split(',')
        labels_to_names[int(data[1])] = data[0]
f.close()

In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# # show images inline
# %matplotlib inline

# # automatically reload modules when they have changed
# %load_ext autoreload
# %autoreload 2

# img_name = '402_frame_5953.jpg'
# img_path = '/project/ibm/a2301133/402/'
# img = cv2.imread(img_path + img_name)
# plt.imshow(img)
# plt.show()

In [ ]:
# video_path = '/project/ibm/a2301133/402/'
# video_name = '2018-03-01_14-08-31-402_cam.mp4'
# vidcap = cv2.VideoCapture(video_path + video_name)
# success, image = vidcap.read()
# count = 0
# while success:
#     cv2.imwrite(video_path + "%d.jpg" % count, image)     # save frame as JPEG file      
#     success, image = vidcap.read()
#     print('Read a new frame: ', success)
#     count += 1

## Run detection on example

In [8]:
image_folder = '/project/ibm/a2301133/402/'

In [ ]:

images = os.listdir(image_folder)
images = [x for x in images if '.jpg' in x]
threshold = 0.5
n = len(images)
i = 0
with open("402_result_new.txt", "wb") as f:
    for img_path in images:
        image = read_image_bgr(image_folder + img_path)
        image = preprocess_image(image)
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        n = len(scores)
        m = len(scores[0])
        for i in xrange(n):
            for j in xrange(m):
                if scores[i][j] >= threshold:
                    xmin, ymin, xmax, ymax = boxes[i][j][0], boxes[i][j][1], boxes[i][j][2], boxes[i][j][3]
                    bbox = str(xmin) + ',' + str(ymin) + ',' + str(xmax) + ',' + str(ymax)
                    f.write(str(img_path) +',' + str(bbox) + ',' + str(labels[i][j]) + ',' + str(scores[i][j]) + '\n')
        i += 1
f.close()

In [ ]:
import json
frames_list = []
txt_file = open("402_result_new.txt" , 'rb')
output_path = "402_result_new.json"
for line in txt_file:
    line = line.strip().split(',')
    frame_num = line[0]
    f_dict = {'frame_number': str(frame_num), 'RoIs': ''}
    xmin, ymin, xmax, ymax = int(float(line[1])), int(float(line[2])), int(float(line[3])), int(float(line[4]))
    delta_x, delta_y = xmax - xmin, ymax - ymin
    xmin, ymin, delta_x, delta_y = str(xmin), str(ymin), str(delta_x), str(delta_y)
    rois_list = [xmin, ymin, delta_x, delta_y, labels_to_names.get(int(line[5]))]
    if len(rois_list) > 0:
        f_dict['RoIs'] = ','.join(rois_list) + ';'
    frames_list.append(f_dict)

json_dict = {'output': {'frames': frames_list}}
with open(output_path, 'wb') as out_file:
    json.dump(json_dict, out_file, indent = 4)


In [ ]:
len(data)

In [5]:
data = os.listdir(image_folder)
data = [x for x in data if '.jpg' in x]
data.sort()
# data = pd.read_csv("198_result_new.txt", header = None)
# data = ['0009660.jpg']

for i in range(0, len(data)):
    image = read_image_bgr(image_folder + data[i])
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis = 0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)

    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.margins(0,0)
    plt.imshow(draw)
    plt.savefig(image_folder + 'FasterRCNN/' + data[i], transparent=True, dpi=300, pad_inches = 0, bbox_inches = 'tight')
#     plt.show()
    print data[i]
    plt.close()

('processing time: ', 3.084460973739624)
0.jpg
('processing time: ', 0.15691804885864258)
1.jpg
('processing time: ', 0.1518700122833252)
10.jpg
('processing time: ', 0.14286494255065918)
100.jpg
('processing time: ', 0.1422581672668457)
1000.jpg
('processing time: ', 0.14442205429077148)
1001.jpg
('processing time: ', 0.1437969207763672)
1002.jpg
('processing time: ', 0.1418311595916748)
1003.jpg
('processing time: ', 0.14138197898864746)
1004.jpg
('processing time: ', 0.14291691780090332)
1005.jpg
('processing time: ', 0.1416919231414795)
1006.jpg
('processing time: ', 0.14346098899841309)
1007.jpg
('processing time: ', 0.14332294464111328)
1008.jpg
('processing time: ', 0.151900053024292)
1009.jpg
('processing time: ', 0.14349985122680664)
101.jpg
('processing time: ', 0.14353299140930176)
1010.jpg
('processing time: ', 0.14454412460327148)
1011.jpg
('processing time: ', 0.13596200942993164)
1012.jpg
('processing time: ', 0.14840292930603027)
1013.jpg
('processing time: ', 0.1448500

114.jpg
('processing time: ', 0.14274907112121582)
1140.jpg
('processing time: ', 0.14128613471984863)
1141.jpg
('processing time: ', 0.14742088317871094)
1142.jpg
('processing time: ', 0.14899301528930664)
1143.jpg
('processing time: ', 0.13876891136169434)
1144.jpg
('processing time: ', 0.13737702369689941)
1145.jpg
('processing time: ', 0.13822603225708008)
1146.jpg
('processing time: ', 0.13533520698547363)
1147.jpg
('processing time: ', 0.1361079216003418)
1148.jpg
('processing time: ', 0.13936901092529297)
1149.jpg
('processing time: ', 0.13858795166015625)
115.jpg
('processing time: ', 0.1496131420135498)
1150.jpg
('processing time: ', 0.13956809043884277)
1151.jpg
('processing time: ', 0.14273595809936523)
1152.jpg
('processing time: ', 0.13840103149414062)
1153.jpg
('processing time: ', 0.13878798484802246)
1154.jpg
('processing time: ', 0.1367659568786621)
1155.jpg
('processing time: ', 0.14072895050048828)
1156.jpg
('processing time: ', 0.1393120288848877)
1157.jpg
('process

1283.jpg
('processing time: ', 0.14415597915649414)
1284.jpg
('processing time: ', 0.14832806587219238)
1285.jpg
('processing time: ', 0.14032292366027832)
1286.jpg
('processing time: ', 0.14093494415283203)
1287.jpg
('processing time: ', 0.13931584358215332)
1288.jpg
('processing time: ', 0.14535307884216309)
1289.jpg
('processing time: ', 0.14297008514404297)
129.jpg
('processing time: ', 0.13843107223510742)
1290.jpg
('processing time: ', 0.1414480209350586)
1291.jpg
('processing time: ', 0.14160609245300293)
1292.jpg
('processing time: ', 0.14812803268432617)
1293.jpg
('processing time: ', 0.14432120323181152)
1294.jpg
('processing time: ', 0.13805699348449707)
1295.jpg
('processing time: ', 0.14120984077453613)
1296.jpg
('processing time: ', 0.13697218894958496)
1297.jpg
('processing time: ', 0.14420700073242188)
1298.jpg
('processing time: ', 0.1374971866607666)
1299.jpg
('processing time: ', 0.1505420207977295)
13.jpg
('processing time: ', 0.15929388999938965)
130.jpg
('processi

('processing time: ', 0.13910412788391113)
1427.jpg
('processing time: ', 0.13858819007873535)
1428.jpg
('processing time: ', 0.13924288749694824)
1429.jpg
('processing time: ', 0.14106202125549316)
143.jpg
('processing time: ', 0.1429119110107422)
1430.jpg
('processing time: ', 0.13967108726501465)
1431.jpg
('processing time: ', 0.13948583602905273)
1432.jpg
('processing time: ', 0.13918685913085938)
1433.jpg
('processing time: ', 0.13922500610351562)
1434.jpg
('processing time: ', 0.1433100700378418)
1435.jpg
('processing time: ', 0.14032793045043945)
1436.jpg
('processing time: ', 0.14079594612121582)
1437.jpg
('processing time: ', 0.14071297645568848)
1438.jpg
('processing time: ', 0.14809513092041016)
1439.jpg
('processing time: ', 0.13790297508239746)
144.jpg
('processing time: ', 0.13840198516845703)
1440.jpg
('processing time: ', 0.14209294319152832)
1441.jpg
('processing time: ', 0.13996410369873047)
1442.jpg
('processing time: ', 0.1385970115661621)
1443.jpg
('processing time

157.jpg
('processing time: ', 0.13924884796142578)
1570.jpg
('processing time: ', 0.13427186012268066)
1571.jpg
('processing time: ', 0.13859200477600098)
1572.jpg
('processing time: ', 0.13565993309020996)
1573.jpg
('processing time: ', 0.14346003532409668)
1574.jpg
('processing time: ', 0.13583803176879883)
1575.jpg
('processing time: ', 0.13314104080200195)
1576.jpg
('processing time: ', 0.13662004470825195)
1577.jpg
('processing time: ', 0.14005708694458008)
1578.jpg
('processing time: ', 0.14018893241882324)
1579.jpg
('processing time: ', 0.13731789588928223)
158.jpg
('processing time: ', 0.14327502250671387)
1580.jpg
('processing time: ', 0.13159394264221191)
1581.jpg
('processing time: ', 0.13568782806396484)
1582.jpg
('processing time: ', 0.14808106422424316)
1583.jpg
('processing time: ', 0.13717889785766602)
1584.jpg
('processing time: ', 0.1464221477508545)
1585.jpg
('processing time: ', 0.14091181755065918)
1586.jpg
('processing time: ', 0.13727283477783203)
1587.jpg
('proc

...

7723.jpg
('processing time: ', 0.13558697700500488)
7724.jpg
('processing time: ', 0.13545799255371094)
7725.jpg
('processing time: ', 0.13645005226135254)
7726.jpg
('processing time: ', 0.14168810844421387)
7727.jpg
('processing time: ', 0.14737486839294434)
7728.jpg
('processing time: ', 0.14541006088256836)
7729.jpg
('processing time: ', 0.1403028964996338)
773.jpg
('processing time: ', 0.14429020881652832)
7730.jpg
('processing time: ', 0.14455008506774902)
7731.jpg
('processing time: ', 0.13321590423583984)
7732.jpg
('processing time: ', 0.14231109619140625)
7733.jpg
('processing time: ', 0.14680719375610352)
7734.jpg
('processing time: ', 0.14595818519592285)
7735.jpg
('processing time: ', 0.13573980331420898)
7736.jpg
('processing time: ', 0.1368269920349121)
7737.jpg
('processing time: ', 0.14518404006958008)
7738.jpg
('processing time: ', 0.15211796760559082)
7739.jpg
('processing time: ', 0.1347038745880127)
774.jpg
('processing time: ', 0.14513707160949707)
7740.jpg
('proces

7867.jpg
('processing time: ', 0.14931321144104004)
7868.jpg
('processing time: ', 0.14558911323547363)
7869.jpg
('processing time: ', 0.14510798454284668)
787.jpg
('processing time: ', 0.1369340419769287)
7870.jpg
('processing time: ', 0.14594793319702148)
7871.jpg
('processing time: ', 0.13899683952331543)
7872.jpg
('processing time: ', 0.13657402992248535)
7873.jpg
('processing time: ', 0.14708685874938965)
7874.jpg
('processing time: ', 0.14130401611328125)
7875.jpg
('processing time: ', 0.13990306854248047)
7876.jpg
('processing time: ', 0.13401293754577637)
7877.jpg
('processing time: ', 0.1411120891571045)
7878.jpg
('processing time: ', 0.14104914665222168)
7879.jpg
('processing time: ', 0.1438751220703125)
788.jpg
('processing time: ', 0.1416921615600586)
7880.jpg
('processing time: ', 0.1514298915863037)
7881.jpg
('processing time: ', 0.13810300827026367)
7882.jpg
('processing time: ', 0.14016389846801758)
7883.jpg
('processing time: ', 0.14009308815002441)
7884.jpg
('processi

('processing time: ', 0.14438390731811523)
814.jpg
('processing time: ', 0.1425788402557373)
815.jpg
('processing time: ', 0.13493108749389648)
816.jpg
('processing time: ', 0.137955904006958)
817.jpg
('processing time: ', 0.14046096801757812)
818.jpg
('processing time: ', 0.14603900909423828)
819.jpg
('processing time: ', 0.1398460865020752)
82.jpg
('processing time: ', 0.14339900016784668)
820.jpg
('processing time: ', 0.1490950584411621)
821.jpg
('processing time: ', 0.13958311080932617)
822.jpg
('processing time: ', 0.1364290714263916)
823.jpg
('processing time: ', 0.13274192810058594)
824.jpg
('processing time: ', 0.14351201057434082)
825.jpg
('processing time: ', 0.14566397666931152)
826.jpg
('processing time: ', 0.13870501518249512)
827.jpg
('processing time: ', 0.13805198669433594)
828.jpg
('processing time: ', 0.1422710418701172)
829.jpg
('processing time: ', 0.1388230323791504)
83.jpg
('processing time: ', 0.14365816116333008)
830.jpg
('processing time: ', 0.14289379119873047

960.jpg
('processing time: ', 0.13862395286560059)
961.jpg
('processing time: ', 0.14548993110656738)
962.jpg
('processing time: ', 0.13894200325012207)
963.jpg
('processing time: ', 0.13846516609191895)
964.jpg
('processing time: ', 0.1401209831237793)
965.jpg
('processing time: ', 0.1504969596862793)
966.jpg
('processing time: ', 0.13793683052062988)
967.jpg
('processing time: ', 0.15349698066711426)
968.jpg
('processing time: ', 0.1455690860748291)
969.jpg
('processing time: ', 0.13541102409362793)
97.jpg
('processing time: ', 0.13823390007019043)
970.jpg
('processing time: ', 0.15050101280212402)
971.jpg
('processing time: ', 0.1436760425567627)
972.jpg
('processing time: ', 0.14602208137512207)
973.jpg
('processing time: ', 0.14964604377746582)
974.jpg
('processing time: ', 0.15371394157409668)
975.jpg
('processing time: ', 0.14317107200622559)
976.jpg
('processing time: ', 0.14256000518798828)
977.jpg
('processing time: ', 0.1435239315032959)
978.jpg
('processing time: ', 0.13940

IOError: [Errno 2] No such file or directory: '/project/ibm/a2301133/402/test.jpg'

In [9]:
import cv2
import os

result_folder = image_folder + 'FasterRCNN/'
video_name = '402.avi'

images = [img for img in os.listdir(result_folder) if img.endswith(".jpg")]
images = sorted(images, key = lambda x: int(x[:-4]))
frame = cv2.imread(os.path.join(result_folder, images[0]))
height, width, layers = frame.shape
frame = cv2.resize(frame, (width,height))
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video = cv2.VideoWriter(result_folder + video_name, fourcc, 10, (width,height))

for image in images:
    current_image = cv2.imread(os.path.join(result_folder, image))
    video.write(current_image)

cv2.destroyAllWindows()
video.release()